This notebook reads GSL v0 with labels and BERT predictions, then hydrate with all query and listing features

In [1]:
import pandas as pd
import numpy as np
import time
import json
import re
from copy import deepcopy
from google.cloud import bigquery

In [2]:
df = pd.read_excel("./data/gsl_eval_v0_internal_label_bert_preds.xlsx")

print(df.shape)
print(df.columns)

(1200, 30)
Index(['query', 'queryEn', 'listingId', 'titleEn_vertica', 'etsyUUID',
       'platform', 'userLanguage', 'anno_data_source',
       'labelbox_majority_label', 'label_annotator_1', 'label_annotator_2',
       'label_annotator_3', 'is_gsl_v0_eval', 'v2_bert_pred_labels',
       'v2_bert_score_not_relevant', 'v2_bert_score_prob_partial',
       'v2_bert_score_relevant', 'label_etsy_1', 'label_etsy_2',
       'label_etsy_3', 'etsy_person_1', 'etsy_person_2', 'etsy_person_3',
       'etsy_notes', 'queryIsGift', 'labelbox_round_label',
       'etsy_majority_label', 'etsy_round_label', 'etsy_unanimous',
       'labelbox_unanimous'],
      dtype='object')


In [3]:
client = bigquery.Client(project="etsy-search-ml-dev")

us_v2_query = """WITH query_listing_pairs AS (
  SELECT
     etsyUUID, query, listingId, 
     listingTitle,
     listingDescription,
     listingTaxo,
     listingImageUrls
  FROM `etsy-sr-etl-prod.yzhang.sem_rel_human_annotation_v2`
),
listing_fb AS (
  SELECT 
    key as listingId,
    IFNULL(verticaListings_tags, "") listingTags,
    (SELECT STRING_AGG(element, ';') FROM UNNEST(kbAttributesV2_sellerAttributesV2.list)) listingAttributes,
    IFNULL(verticaSellerBasics_shopName, "") listingShopName,
    (SELECT STRING_AGG(element, ', ') FROM UNNEST(descNgrams_ngrams.list)) listingDescNgrams,
    IFNULL(listingLlmFeatures_llmHeroImageDescription, "") listingHeroImageCaption,
  FROM `etsy-ml-systems-prod.feature_bank_v2.listing_feature_bank_2025-02-19`
),
grouped_variation_values AS (
  SELECT 
    listing_id, 
    attribute_name,
    STRING_AGG(attribute_value, ', ') AS grouped_attributes
  FROM `etsy-data-warehouse-prod.listing_mart.listing_variation_attributes`
  GROUP BY listing_id, attribute_name
),
listing_variations AS (
  SELECT 
    listing_id as listingId,
    STRING_AGG(CONCAT(attribute_name, ': ', grouped_attributes), '; ') AS listingVariations
  FROM grouped_variation_values
  GROUP BY listing_id
),
review_raw AS (
  SELECT listing_id, review, DATE(TIMESTAMP_SECONDS(update_date)) review_last_update_date, 
  FROM `etsy-data-warehouse-prod.etsy_shard.listing_review`
  WHERE review IS NOT null AND review != ""
),
recent_five_reviews AS (
  SELECT *
  FROM review_raw
  QUALIFY ROW_NUMBER() OVER(PARTITION BY listing_id ORDER BY review_last_update_date DESC) <= 5
),
listing_reviews AS (
  SELECT
    listing_id AS listingId, 
    STRING_AGG(review, ' | ') listingReviews
  FROM recent_five_reviews
  GROUP BY listing_id
),
query_rewrites AS (
  SELECT key AS query, STRING_AGG(unnested_value, ", ") AS queryRewrites
  FROM `etsy-search-ml-dev.mission_understanding.smu_query_rewriting_v2_dpo_semrel`, 
    UNNEST(value) AS unnested_value
  GROUP BY key
),
qe_raw AS (
  SELECT DISTINCT
    searchQuery AS query,
    entities AS queryEntities
  FROM `etsy-data-warehouse-prod.arizona.query_entity_features`
),
query_entities AS (
  SELECT * 
  FROM qe_raw
  QUALIFY ROW_NUMBER() OVER(PARTITION BY query ORDER BY rand()) = 1
)
SELECT 
  etsyUUID, 
  query, 
  listingId, 
  queryRewrites,
  queryEntities,
  listingTitle, "" AS listingTitleEn,
  listingTaxo,
  listingTags,
  listingAttributes,
  listingShopName,
  listingDescription, "" AS listingDescriptionEn,
  listingDescNgrams,
  listingImageUrls,
  listingHeroImageCaption,
  listingVariations,
  listingReviews,
FROM query_listing_pairs
LEFT JOIN listing_fb USING (listingId)
LEFT JOIN listing_variations USING (listingId)
LEFT JOIN listing_reviews USING (listingId)
LEFT JOIN query_rewrites USING (query)
LEFT JOIN query_entities USING (query)
"""
us_job = client.query(us_v2_query)
us_features_df = us_job.result().to_dataframe()

/Users/yzhang/development/etsy-llm/.venv/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
I0000 00:00:1740091737.482769 3128140 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [4]:
intl_query = """WITH query_listing_pairs AS (
  SELECT
    etsyUUID, query, listingId, 
    queryEn,
    listingTitle, listingTitleEn,
    listingDescription, listingDescriptionEn,
    listingAttributes,
    listingShopName,
    listingTaxo,
    listingTags,
  FROM `etsy-search-ml-dev.aclapp.isearch_semrel_surveyv1_teacherv2_features_w_primaryLang`
),
listing_fb AS (
  SELECT 
    key as listingId,
    (SELECT STRING_AGG(element, ', ') FROM UNNEST(descNgrams_ngrams.list)) listingDescNgrams,
    IFNULL(listingLlmFeatures_llmHeroImageDescription, "") listingHeroImageCaption,
  FROM `etsy-ml-systems-prod.feature_bank_v2.listing_feature_bank_2025-02-19`
),
grouped_variation_values AS (
  SELECT 
    listing_id, 
    attribute_name,
    STRING_AGG(attribute_value, ', ') AS grouped_attributes
  FROM `etsy-data-warehouse-prod.listing_mart.listing_variation_attributes`
  GROUP BY listing_id, attribute_name
),
listing_variations AS (
  SELECT 
    listing_id as listingId,
    STRING_AGG(CONCAT(attribute_name, ': ', grouped_attributes), '; ') AS listingVariations
  FROM grouped_variation_values
  GROUP BY listing_id
),
review_raw AS (
  SELECT listing_id, review, DATE(TIMESTAMP_SECONDS(update_date)) review_last_update_date, 
  FROM `etsy-data-warehouse-prod.etsy_shard.listing_review`
  WHERE review IS NOT null AND review != ""
),
recent_five_reviews AS (
  SELECT *
  FROM review_raw
  QUALIFY ROW_NUMBER() OVER(PARTITION BY listing_id ORDER BY review_last_update_date DESC) <= 5
),
listing_reviews AS (
  SELECT
    listing_id AS listingId, 
    STRING_AGG(review, ' | ') listingReviews
  FROM recent_five_reviews
  GROUP BY listing_id
),
listing_images AS (
  SELECT 
    listing_id listingId,
    STRING_AGG(url, ';' ORDER BY img_rank ASC) listingImageUrls
  FROM `etsy-data-warehouse-prod.computer_vision.listing_image_paths`
  GROUP BY listing_id
),
query_rewrites AS (
  SELECT key AS query, STRING_AGG(unnested_value, ", ") AS queryRewrites
  FROM `etsy-search-ml-dev.mission_understanding.smu_query_rewriting_v2_dpo_semrel`, 
    UNNEST(value) AS unnested_value
  GROUP BY key
),
qe_raw AS (
  SELECT DISTINCT
    searchQuery AS query,
    entities AS queryEntities
  FROM `etsy-data-warehouse-prod.arizona.query_entity_features`
),
query_entities AS (
  SELECT * 
  FROM qe_raw
  QUALIFY ROW_NUMBER() OVER(PARTITION BY query ORDER BY rand()) = 1
)
SELECT 
  etsyUUID, 
  query, 
  listingId, 
  queryRewrites,
  queryEntities,
  listingTitle, listingTitleEn,
  listingTaxo,
  listingTags,
  listingAttributes,
  listingShopName,
  listingDescription, listingDescriptionEn,
  listingDescNgrams,
  listingImageUrls,
  listingHeroImageCaption,
  listingVariations,
  listingReviews,
FROM query_listing_pairs
LEFT JOIN listing_fb USING (listingId)
LEFT JOIN listing_variations USING (listingId)
LEFT JOIN listing_reviews USING (listingId)
LEFT JOIN listing_images USING (listingId)
LEFT JOIN query_rewrites USING (query)
LEFT JOIN query_entities USING (query)
""" 
intl_job = client.query(intl_query)
intl_features_df = intl_job.result().to_dataframe()

I0000 00:00:1740091800.387238 3128140 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [5]:
features_df = pd.concat([us_features_df, intl_features_df], ignore_index=True)
features_df.shape

(56793, 18)

### Cleaning

In [6]:
def clean_query_entities(qe_str):
    # process json formatted strings into natual text strings
    qe_data = json.loads(qe_str)
    qe_output = ""
    for k, v in qe_data.items():
        if len(v) > 0:
            qe_output += f"{k}:{','.join(v)};"
    qe_output = qe_output.strip()
    return qe_output


def clean_tags(tag_str):
    # remove leading dot
    if tag_str.startswith("."):
        tag_str = tag_str[1:]
    # replace dots with comma because dots cannot be saved to excel
    output = tag_str.replace(".", ", ")
    return output

    
def normalize_strings(s):
    s = re.sub(r"&gt;", ">", s)
    s = re.sub(r"&lt;", "<", s)
    s = re.sub(r"&#39;|‘|’", "'", s)
    s = re.sub(r"&quot;|“|”|''", '"', s)
    s = re.sub(r"\x00", "", s)
    return s

In [7]:
features_df.fillna("", inplace=True)

In [8]:
# clean query entities - transform json format into texts
features_df["queryEntities"] = features_df["queryEntities"].apply(lambda x: "" if x == "" else clean_query_entities(x))
# clean attributes - : for separator instead of #, to lower case
features_df["listingAttributes"] = features_df["listingAttributes"].str.replace("#",':').str.lower()
# clean tags - replace . with ,
features_df["listingTags"] = features_df["listingTags"].apply(clean_tags)

In [9]:
features_df["listingTitle"] = features_df["listingTitle"].apply(normalize_strings)
features_df["listingTitleEn"] = features_df["listingTitleEn"].apply(normalize_strings)
features_df["listingTags"] = features_df["listingTags"].apply(normalize_strings)
features_df["listingDescription"] = features_df["listingDescription"].apply(normalize_strings)
features_df["listingDescriptionEn"] = features_df["listingDescriptionEn"].apply(normalize_strings)
features_df["listingDescNgrams"] = features_df["listingDescNgrams"].apply(normalize_strings)
features_df["listingVariations"] = features_df["listingVariations"].apply(normalize_strings)
features_df["listingReviews"] = features_df["listingReviews"].apply(normalize_strings)

### Join features with query listing pairs

In [10]:
merged_df = pd.merge(df, features_df, on=["etsyUUID", "query", "listingId"], how="left")

In [11]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 45 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   query                       1200 non-null   object 
 1   queryEn                     544 non-null    object 
 2   listingId                   1200 non-null   int64  
 3   titleEn_vertica             1200 non-null   object 
 4   etsyUUID                    1200 non-null   object 
 5   platform                    1200 non-null   object 
 6   userLanguage                1200 non-null   object 
 7   anno_data_source            1200 non-null   object 
 8   labelbox_majority_label     1200 non-null   object 
 9   label_annotator_1           1200 non-null   object 
 10  label_annotator_2           1200 non-null   object 
 11  label_annotator_3           1200 non-null   object 
 12  is_gsl_v0_eval              1200 non-null   bool   
 13  v2_bert_pred_labels         1200 

In [12]:
merged_df.fillna("", inplace=True)

In [25]:
merged_df.to_excel("./data/gsl_eval_v0_all_except_llm.xlsx", index=False)